<a href="https://colab.research.google.com/github/finale22/NLP_study/blob/main/Text_Classification_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 로드 및 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/DL/')

In [ ]:
# IMDB 원본 데이터 전처리
import os

imdb_dir = './aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = [] # y
texts = []  # x

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)                 # 분류 카테고리 별 폴더 경로
    if label_type == 'neg':
        label = 0
    else:
        label = 1
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf8') # 파일을 하나씩 읽기
            texts.append(f.read())                                   # texts 리스트에 추가
            f.close()
            labels.append(label)                                     # labels 리스트에 추가

In [ ]:
print("text data")
for i in range(5):
    print(f"{texts[i]}: {labels[i]}")

# 2. 데이터 토큰화

In [ ]:
# 텍스트 데이터 토큰화
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequence
import numpy as np

maxlen = 100                     # 토큰 개수 맞추기
training_samples = 200           # 훈련 샘플 개수
validation_samples = 10000       # 검증 샘플 개수
max_words = 10000                # 데이터셋에서 가장 빈도 높은 10000개의 단어만 사용

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print(f"고유 토큰 개수: {len(word_index)}")

data = pad_sequence(sequences, maxlen=maxlen) # padding
labels = np.asarray(labels)
print(f"데이터 크기: {data.shape}")
print(f"레이블 크기: {labels.shape}")

indices = np.arange(data.shape[0]) # 데이터 분할 전 섞기 위한 작업
np.random.shuffle(indices)         # 섞기
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = label[:training_samples]
x_val = data[training_samples:training_samples + validation_samples]
y_val = label[training_samples:training_samples + validation_samples]

# 3. GloVe 임베딩 전처리

In [ ]:
glove_dir = './'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encodin='utf8')
for line in f:
    values = line.split()
    word = value[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print(f"단어 벡터 개수: {len(embeddings_index)}")

In [ ]:
# GloVe 단어 임베딩 행렬 준비
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # 임베딩 인덱스에 없는 단어는 모두 0으로 처리

# 4. 모델 정의

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# 5. 모델에 GloVe 임베딩 로드

In [ ]:
# 사전 훈련된 단어 임베딩을 Embedding layer에 로드
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# 6. 모델 학습

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
hist = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

In [ ]:
# 그래프
import matplotlib.pyplot as plt

history = hist.history
acc = history['acc']
val_acc = history['val_acc']
loss = history['loss']
val_loss = history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# 7. 사전 훈련된 단어 임베딩을 사용하지 않고 모델 학습

In [ ]:
# 사전 훈련된 단어 임베딩을 사용하지 않고 같은 모델 학습하기
# 사전 훈련된 단어 임베딩을 사용하여 학습하는 것과 사용하지 않고 학습하는 것을 비교
from keras.backends import clear_session
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

clear_session()
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
hist = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

In [ ]:
# 그래프
import matplotlib.pyplot as plt

history = hist.history
acc = history['acc']
val_acc = history['val_acc']
loss = history['loss']
val_loss = history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# 8. 모델 평가

In [ ]:
# 테스트 데이터 전처리

train_dir = os.path.join(imdb_dir, 'test')

labels = [] # y
texts = []  # x

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)                 # 분류 카테고리 별 폴더 경로
    if label_type == 'neg':
        label = 0
    else:
        label = 1
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf8') # 파일을 하나씩 읽기
            texts.append(f.read())                                   # texts 리스트에 추가
            f.close()
            labels.append(label)                                     # labels 리스트에 추가

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [ ]:
model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)